In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [3]:
class MyclassificationHyperModel(kt.HyperModel):
    def build(self,hp):
        n_hidden=hp.Int("n_hidden",min_value=0,max_value=8,default=2)
        n_neurons=hp.Int("n_neurons",min_value=16,max_value=256)
        learning_rate=hp.Float("learning_rate",min_value=1e-4,max_value=1e-2,sampling="log")
        optimizer=hp.Choice("optimizer",values=["sgd","adam"])

        if optimizer=="sgd":
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate)
        else:
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

        model=tf.keras.Sequential()
        model.add(tf.keras.layers.Flatten())
        for _ in range(n_hidden):
            model.add(tf.keras.layers.Dense(n_neurons,activation='relu'))  

        model.add(tf.keras.layers.Dense(10,activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=["accuracy"])
        return model

    def fit(self, hp, model, *args, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer=tf.keras.layers.Normalization()
            X=norm_layer(X)

        return model.fit(*args,**kwargs)              

USING HYPERBAND 

In [4]:
hyperband_tuner=kt.Hyperband(
    MyclassificationHyperModel(),objective="val_accuracy",seed=42,max_epochs=10,factor=3,hyperband_iterations=2,overwrite=True,
    directory="my_fashion_mnist",project_name="hyperband"
)

USING BAYESIAN OPTIMIZATION

In [5]:
bayesian_opt_tuner=kt.BayesianOptimization(
    MyclassificationHyperModel(),objective="val_accuracy",seed=42,max_trials=10,alpha=1e-4,beta=2.6,
    overwrite=True,directory="my_fashion_mnist",project_name="bayesian_opt"
)